In [ ]:
# -----------------------------
# 1️⃣ Imports
# -----------------------------
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification
import shap
from lime.lime_text import LimeTextExplainer

# -----------------------------
# 2️⃣ Load model and tokenizer
# -----------------------------
model_name = "./amharic_ner_model_final"  # replace with your path
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)
model.eval()

# -----------------------------
# 3️⃣ Prediction function for SHAP/LIME
# -----------------------------
def predict_probabilities(texts):
    if isinstance(texts, str):
        texts = [texts]
    elif isinstance(texts, np.ndarray):
        texts = texts.tolist()
    elif isinstance(texts, list):
        texts = [str(t) for t in texts]
    else:
        raise ValueError("Input must be str, list of str, or numpy array of str")
    
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs).logits  # [batch, seq_len, num_labels]
    
    probs = torch.softmax(outputs, dim=-1).cpu().numpy()
    # Flatten seq_len and take max probability per token
    probs_summary = probs.max(axis=1)
    return probs_summary

# -----------------------------
# 4️⃣ Sample text
# -----------------------------
sample_text = "አዲስ አበባ ቢሮ ቁ 123 ላይ ነው"

# -----------------------------
# 5️⃣ SHAP Explanation
# -----------------------------
print("Running SHAP explanation...")
shap_explainer = shap.Explainer(predict_probabilities, masker=shap.maskers.Text(tokenizer))
shap_values = shap_explainer([sample_text])
shap.plots.text(shap_values[0])  # visualizes in notebook or script

# -----------------------------
# 6️⃣ LIME Explanation
# -----------------------------
print("\nRunning LIME explanation...")
lime_explainer = LimeTextExplainer(class_names=[str(i) for i in range(model.config.num_labels)])
lime_exp = lime_explainer.explain_instance(
    sample_text,
    predict_probabilities,
    num_features=10
)

# Print in console
print("LIME Explanation (feature weights):")
for feature, weight in lime_exp.as_list():
    print(f"{feature}: {weight:.6f}")

# Save to HTML
with open("lime_explanation.html", "w", encoding="utf-8") as f:
    f.write(lime_exp.as_html())
print("LIME explanation saved as lime_explanation.html")

# -----------------------------
# ✅ Done
# -----------------------------
print("\n✅ Combined SHAP + LIME analysis complete!")



Running SHAP explanation...



Running LIME explanation...
LIME Explanation (feature weights):
አበባ: -0.002268
ቢሮ: 0.001454
ነው: 0.001058
አዲስ: 0.000931
ላይ: -0.000406
123: -0.000349
ቁ: -0.000150
LIME explanation saved as lime_explanation.html

✅ Combined SHAP + LIME analysis complete!


TypeError: text() got an unexpected keyword argument 'show'

<Figure size 1000x300 with 0 Axes>